In [1]:
import time
import json
import os
import re

from jupyter_core.migrate import regex

from LLM import DeepseekChat
from utils import ReadFiles
from tqdm import tqdm

In [2]:
# file_path = './data/test.txt'
file_path = './data/甄嬛传_test.txt'
docs = ReadFiles(file_path).get_content(max_token_len=300, cover_content=0)

In [3]:
len(docs), docs[1]

(12,
 '第一章 风波。这场选秀对我的意义并不大，我只不过来转一圈充个数便回去。爹爹说，我们的女儿娇纵惯了，怎受得了宫廷约束。罢了罢了，平平安安嫁个好郎君也就是了。娘总说像我女儿这般容貌家世，更不肖说人品才学一定要给我挑最好的郎君。我也一直是这样想的，我甄嬛一定要嫁这世间上最好的男儿，和他结成连理平平安安白首到老，便是幸福了。我不能轻易辜负了自己。而皇帝坐拥天下，却未必是我心中认可的最好的男儿。至少，他不能专心待我。因而，我并不细心打扮。脸上薄施粉黛，一身浅绿色裙装。头上斜簪一朵新摘的白芙蓉，除此之外只挽一支碧玉七宝玲珑簪，以彰显我是吏部侍郎的女儿，并非一般的小家碧玉，可以轻易小瞧了我去。如此轻描淡写，我只需等着皇上“撂牌子”，让我落选。')

In [4]:
import importlib
import extract
importlib.reload(extract)
# sys_prompt = system_prompt(novel_schema)
sys_prompt = extract.SYSTEM_PROMPT_CHINESE
print(sys_prompt)


## 任务描述
你是一位专业的文本信息提取专家，你的任务是从用户输入的剧本小说中提取角色的台词对话以及对话所属的人物名称。


## 要求
1. 分析文本内容，提取出文本中的人物的对话台词以及台词对应的人物名称。
2. 你只能提取 中文双引号“” 括起来的内容作为对话台词内容，禁止提取文本的其它旁白内容，禁止胡乱编造。
3. 输出格式为json列表，参照【输出格式示例】，列表的每个元素为字典，字典包含：role，dialogue 两个字段。role为角色姓名，dialogue为对话台词。
4. 如果文本中没有明确的对话台词，则返回josn格式的空列表[]


## 输入格式示例
'
第一章 风波。 这场选秀对我的意义并不大，我只不过来转一圈充个数便回去。爹爹说，我们的女儿娇纵惯了，怎受得了宫廷约束。
因而，我并不细心打扮。脸上薄施粉黛，一身浅绿色裙装。
满满一屋子秀女，与我相熟的只有济州都督沈自山的女儿沈眉庄。我府与她京中外婆府上比邻而居，我和她更是自小一起长大，情谊非寻常可比。她远远看见我，走过来的执我的手，面含喜色关切道：“嬛儿，你在这里我就放心了。上次听外婆说妹妹受了风寒，可大好了？”
我急忙起身说：“不过是咳嗽了两声，早就好了。劳姐姐费心。路上颠簸，姐姐可受了风尘之苦。”
她点点头，细细看我两眼，微笑说：“在京里休息了两日，已经好得多。妹妹今日打扮得好素净，益发显得姿容出众，卓而不群。”
 我脸上飞红，害羞道：“姐姐不是美人么？这样说岂不是要羞煞我。”
'


## 输出格式示例
[
    {"role": "沈眉庄", "dialogue": "嬛儿，你在这里我就放心了。上次听外婆说妹妹受了风寒，可大好了？"},
    {"role": "我", "dialogue": "不过是咳嗽了两声，早就好了。劳姐姐费心。路上颠簸，姐姐可受了风尘之苦。"},
    {"role": "沈眉庄", "dialogue":"在京里休息了两日，已经好得多。妹妹今日打扮得好素净，益发显得姿容出众，卓而不群。"},
    {"role": "我", "dialogue": "姐姐不是美人么？这样说岂不是要羞煞我。"}
]





In [6]:
goal_text = r"""
第一章 选秀。 这场选秀对我的意义并不大，我只不过来转一圈充个数便回去。爹爹说，我们的女儿娇纵惯了，怎受得了宫廷约束。罢了罢了，平平安安嫁个好郎君也就是了。娘总说像我女儿这般容貌家世，更不肖说人品才学一定要给我挑最好的郎君。我也一直是这样想的，我甄嬛一定要嫁这世间上最好的男儿，和他结成连理平平安安白首到老，便是幸福了。我不能轻易辜负了自己。而皇帝坐拥天下，却未必是我心中认可的最好的男儿。至少，他不能专心待我。因而，我并不细心打扮。脸上薄施粉黛，一身浅绿色裙装。头上斜簪一朵新摘的白芙蓉，除此之外只挽一支碧玉七宝玲珑簪，以彰显我是吏部侍郎的女儿，并非一般的小家碧玉，可以轻易小瞧了我去。如此轻描淡写，我只需等着皇上“撂牌子”，让我落选。"""

In [7]:
try:
    goal_text = docs[6]
    model = DeepseekChat()
    model_response = model.chat(sys_prompt, goal_text).strip("```json").strip("```").strip()
    response = json.loads(model_response)
    print(response)
    print("解析成功")
except Exception as e:
    print(f"模型返回:{model_response}")
    print(f"异常信息:{e}")


[{'role': '安陵容', 'dialogue': '陵容刚才只是想到待会要面见圣驾，心中不安，所以一时失手将茶水洒在林姐姐身上，陵容在这里向姐姐请罪，望姐姐原谅。'}, {'role': '林氏', 'dialogue': '凭你也想你见圣驾？真是异想天开！今日之事要作罢也可，你只需跪下向我叩头请罪。'}, {'role': '我', 'dialogue': '不过是一件衣服，林姐姐莫要生气。妹妹自备了衣服，姐姐到后厢换过即可。今日大选，姐姐这样吵闹怕是会惊动了圣驾，若是龙颜因此而震怒，又岂是你我姐妹可以承担的。况且，即便今日圣驾未惊，若是他日传到他人耳中，也会坏了姐姐贤德的名声。为一件衣服因小失大岂非得不偿失，望姐姐三思。'}]
解析成功


In [6]:
file_name = file_path.split('/')[-1].split('.')[0]

for i in tqdm(range(len(docs))):
    response = model.chat(sys_prompt, docs[i])
    try:
        response = json.loads(response)
        for item in response:
            with open(f'./data/{file_name}.jsonl', 'a', encoding='utf-8') as f:
                json.dump(item, f, ensure_ascii=False)
                f.write('\n')
    except Exception as e:
        print(e)

 55%|█████▌    | 16/29 [06:54<06:27, 29.78s/it]

Expecting value: line 1 column 1 (char 0)


 86%|████████▌ | 25/29 [12:50<02:25, 36.33s/it]

Expecting value: line 1 column 1 (char 0)


100%|██████████| 29/29 [14:08<00:00, 29.25s/it]


# 甄嬛传切分提取测试

In [112]:
file_path = './data/甄嬛传_test.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read().splitlines()
    # content = re.sub(r'\s+', '', cnt)
content = [re.sub(r'\s+', '', cnt) for cnt in content if re.sub(r'\s+', '', cnt) != '']
content = [cnt if cnt.endswith("。") else cnt+"。" for cnt in content]
content

['第一章风波。',
 '我初进宫的那一天，是个非常晴朗的日子。乾元十二年农历八月二十，黄道吉日。站在紫禁城空旷的院落里可以看见无比晴好的天空，蓝澄澄的如一汪碧玉，没有一丝云彩，偶尔有大雁成群结队地飞过。',
 '鸿雁高飞，据说这是一个非常好的预兆。',
 '毓祥门外整整齐齐地排列着无数专送秀女的马车，所有的人都鸦雀无声，保持异常的沉默。我和来自各地的秀女站在一起，黑压压一群人，端的是绿肥红瘦，嫩脸修蛾，脂粉香扑鼻。很少有人说话，只专心照看自己的脂粉衣裳是否周全，或是好奇地偷眼观察近旁的秀女。',
 '选秀是每个官家少女的命运，每三年一选，经过层层选拔，将才貌双全的未婚女子选入皇宫，充实后庭。',
 '这场选秀对我的意义并不大，我只不过来转一圈充个数便回去。爹爹说，我们的女儿娇纵惯了，怎受得了宫廷约束。罢了罢了，平平安安嫁个好郎君也就是了。',
 '娘总说像我女儿这般容貌家世，更不肖说人品才学一定要给我挑最好的郎君。我也一直是这样想的，我甄嬛一定要嫁这世间上最好的男儿，和他结成连理平平安安白首到老，便是幸福了。我不能轻易辜负了自己。',
 '而皇帝坐拥天下，却未必是我心中认可的最好的男儿。至少，他不能专心待我。',
 '因而，我并不细心打扮。脸上薄施粉黛，一身浅绿色裙装。头上斜簪一朵新摘的白芙蓉，除此之外只挽一支碧玉七宝玲珑簪，以彰显我是吏部侍郎的女儿，并非一般的小家碧玉，可以轻易小瞧了我去。',
 '如此轻描淡写，我只需等着皇上“撂牌子”，让我落选。',
 '选看秀女的地点在紫禁城内长春宫的正殿云意殿。秀女分成六人一组，由太监引着进去被选看，其余的则在长春宫的东西暖阁等候。选看很简单，朝皇上皇后叩头，然后站着听候吩咐，皇上或者问哪个人几句话，或者问也不问，谢了恩便可。然后由皇上决定是“撂牌子”还是“留用”。“撂牌子”就是淘汰了，”留用”则是被选中，暂居本家，选吉日即可入宫为妃嫔。',
 '皇上早已大婚，也多内宠。这次的选秀，不过是广选妃嫔充实掖庭，为皇上绵延子嗣。',
 '满满一屋子秀女，与我相熟的只有济州都督沈自山的女儿沈眉庄。我府与她京中外婆府上比邻而居，我和她更是自小一起长大，情谊非寻常可比。她远远看见我，走过来的执我的手，面含喜色关切道：“嬛儿，你在这里我就放心了。上次听外婆说妹妹受了风寒，可大好了？”。',
 '我急忙起身说：“不过是咳嗽了两声，早就好了

In [153]:
chunk_size=500
chunks = []

chunk = ""
for line in content:
    chunk += line
    if len(chunk) >= chunk_size:
        chunks.append(chunk)
        chunk = line
if chunk:
    chunks.append(chunk)

In [154]:
# chunks = ["第一章风波。"+chunk for chunk in chunks]
chunks

['第一章风波。我初进宫的那一天，是个非常晴朗的日子。乾元十二年农历八月二十，黄道吉日。站在紫禁城空旷的院落里可以看见无比晴好的天空，蓝澄澄的如一汪碧玉，没有一丝云彩，偶尔有大雁成群结队地飞过。鸿雁高飞，据说这是一个非常好的预兆。毓祥门外整整齐齐地排列着无数专送秀女的马车，所有的人都鸦雀无声，保持异常的沉默。我和来自各地的秀女站在一起，黑压压一群人，端的是绿肥红瘦，嫩脸修蛾，脂粉香扑鼻。很少有人说话，只专心照看自己的脂粉衣裳是否周全，或是好奇地偷眼观察近旁的秀女。选秀是每个官家少女的命运，每三年一选，经过层层选拔，将才貌双全的未婚女子选入皇宫，充实后庭。这场选秀对我的意义并不大，我只不过来转一圈充个数便回去。爹爹说，我们的女儿娇纵惯了，怎受得了宫廷约束。罢了罢了，平平安安嫁个好郎君也就是了。娘总说像我女儿这般容貌家世，更不肖说人品才学一定要给我挑最好的郎君。我也一直是这样想的，我甄嬛一定要嫁这世间上最好的男儿，和他结成连理平平安安白首到老，便是幸福了。我不能轻易辜负了自己。而皇帝坐拥天下，却未必是我心中认可的最好的男儿。至少，他不能专心待我。因而，我并不细心打扮。脸上薄施粉黛，一身浅绿色裙装。头上斜簪一朵新摘的白芙蓉，除此之外只挽一支碧玉七宝玲珑簪，以彰显我是吏部侍郎的女儿，并非一般的小家碧玉，可以轻易小瞧了我去。',
 '因而，我并不细心打扮。脸上薄施粉黛，一身浅绿色裙装。头上斜簪一朵新摘的白芙蓉，除此之外只挽一支碧玉七宝玲珑簪，以彰显我是吏部侍郎的女儿，并非一般的小家碧玉，可以轻易小瞧了我去。如此轻描淡写，我只需等着皇上“撂牌子”，让我落选。选看秀女的地点在紫禁城内长春宫的正殿云意殿。秀女分成六人一组，由太监引着进去被选看，其余的则在长春宫的东西暖阁等候。选看很简单，朝皇上皇后叩头，然后站着听候吩咐，皇上或者问哪个人几句话，或者问也不问，谢了恩便可。然后由皇上决定是“撂牌子”还是“留用”。“撂牌子”就是淘汰了，”留用”则是被选中，暂居本家，选吉日即可入宫为妃嫔。皇上早已大婚，也多内宠。这次的选秀，不过是广选妃嫔充实掖庭，为皇上绵延子嗣。满满一屋子秀女，与我相熟的只有济州都督沈自山的女儿沈眉庄。我府与她京中外婆府上比邻而居，我和她更是自小一起长大，情谊非寻常可比。她远远看见我，走过来的执我的手，面含喜色关切道：“嬛儿，你在这里我就放心了。上次听外婆说妹妹受了风寒，

# 全部甄嬛传剧本测试

In [177]:
file_path = './data/甄嬛传.txt'

with open(file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

chapter_pattern = re.compile(r'^第[一二三四五六七八九十百零\d]+章\s?.*')
script_dict = {}
current_title = None
current_content = []

for line in lines:
    line = line.strip()
    if not line:
        continue

    if chapter_pattern.match(line):
        if current_title:
            script_dict[current_title] = '\n'.join(current_content)
        current_title = line
        current_content = []
    else:
        current_content.append(line)

# 加上最后一章
if current_title:
    script_dict[current_title] = '\n'.join(current_content)

len(script_dict.keys())

93

In [170]:
split_words = r"---（本卷结束） ---"

file_path = './data/甄嬛传_test.txt'

with open(file_path, 'r', encoding='utf-8') as f:
    content = f.read()

cnts = content.split(split_words)
len(cnts[0])

531759

In [11]:
result = json.loads("[]")
result

[]

# json台词对白的数据清洗

In [10]:
import json
import pandas as pd

In [12]:
data = []
file_path = 'data/output/甄嬛传_21.jsonl'
with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

df = pd.DataFrame(data)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1136 entries, 0 to 1287
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   role      1136 non-null   object
 1   dialogue  1136 non-null   object
dtypes: object(2)
memory usage: 26.6+ KB


In [26]:
filtered_df = df[df["dialogue"].str.contains("…")]
filtered_df

,role,dialogue


In [23]:
df["dialogue"] = df["dialogue"].str.replace(r'\.', '', regex=True).str.replace("“", "").str.replace("”", "").str.replace("…", "")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1288 entries, 0 to 1287
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   role      1288 non-null   object
 1   dialogue  1288 non-null   object
dtypes: object(2)
memory usage: 20.2+ KB


In [29]:
df = df.drop_duplicates(subset=['dialogue'], keep='last')

In [31]:
df.head()

,role,dialogue
0,沈眉庄,嬛儿，你在这里我就放心了。上次听外婆说妹妹受了风寒，可大好了？
1,我,不过是咳嗽了两声，早就好了。劳姐姐费心。路上颠簸，姐姐可受了风尘之苦。
3,她,在京里休息了两日，已经好得多。妹妹今日打扮得好素净，益发显得姿容出众，卓而不群。
4,我,姐姐不是美人么？这样说岂不是要羞煞我。
5,我,几日不见，姐姐出落得越发标致了。皇上看见必定过目不忘。


In [32]:
df.to_json(r"./output.jsonl", orient='records', lines=True, force_ascii=False)

## 角色名称统一

In [35]:
df["role"] = df["role"].str.replace("我", "甄嬛")
df["role"] = df["role"].str.replace("嬛儿", "甄嬛")
df["role"] = df["role"].str.replace("嬛嬛", "甄嬛")

In [36]:
df["role"].value_counts()

role
甄嬛      398
眉庄      112
玄凌       72
陵容       47
槿汐       44
       ... 
奴才        1
姑姑        1
眉目不动      1
小施        1
越听        1
Name: count, Length: 89, dtype: int64